In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive

drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
SHORT_TIME = 'short_time'
LONG_TIME = 'long_time'

SHORT_TIME_OR_LONG_TIME = LONG_TIME


In [4]:
HORIZON = 2 if SHORT_TIME_OR_LONG_TIME == SHORT_TIME else 7

In [5]:
# Dateipfad angeben
file_path = f'/content/drive/My Drive/BA/passantenzaehlung_tagesdaten_features_{SHORT_TIME_OR_LONG_TIME}.csv'

# Datei einlesen
df_day = pd.read_csv(file_path)

# Ersten paar Zeilen anzeigen, um sicherzustellen, dass alles korrekt eingelesen wurde
df_day.head()


,timestamp,temperature,pedestrians_count,location,special_event,holidays,weekday_Friday,weekday_Monday,weekday_Saturday,weekday_Sunday,...,pedestrians_count_7d_lag,train_1,test_1,temperature_forecast_1,train_2,test_2,temperature_forecast_2,train_3,test_3,temperature_forecast_3
0,2020-01-08,6.62,23928.0,Schönbornstraße,False,False,False,False,False,False,...,7285.0,True,False,NaN,True,False,NaN,True,False,NaN
1,2020-01-09,12.11,29521.0,Schönbornstraße,False,False,False,False,False,False,...,40406.0,True,False,NaN,True,False,NaN,True,False,NaN
2,2020-01-10,10.05,30220.0,Schönbornstraße,False,False,True,False,False,False,...,41095.0,True,False,NaN,True,False,NaN,True,False,NaN
3,2020-01-11,5.92,44711.0,Schönbornstraße,False,False,False,False,True,False,...,42096.0,True,False,NaN,True,False,NaN,True,False,NaN
4,2020-01-12,4.48,7309.0,Schönbornstraße,False,False,False,False,False,True,...,7847.0,True,False,NaN,True,False,NaN,True,False,NaN


In [6]:
# one hot encode location

df_day = pd.get_dummies(df_day, columns=['location'])

df_day.columns

Index(['timestamp', 'temperature', 'pedestrians_count', 'special_event',
       'holidays', 'weekday_Friday', 'weekday_Monday', 'weekday_Saturday',
       'weekday_Sunday', 'weekday_Thursday', 'weekday_Tuesday',
       'weekday_Wednesday', 'season_Fall', 'season_Spring', 'season_Summer',
       'season_Winter', 'month_sin', 'month_cos', 'month_1', 'month_2',
       'month_3', 'month_4', 'month_5', 'month_6', 'month_7', 'month_8',
       'month_9', 'month_10', 'month_11', 'month_12',
       'pedestrians_count_7d_lag', 'train_1', 'test_1',
       'temperature_forecast_1', 'train_2', 'test_2', 'temperature_forecast_2',
       'train_3', 'test_3', 'temperature_forecast_3', 'location_Kaiserstraße',
       'location_Schönbornstraße', 'location_Spiegelstraße'],
      dtype='object')

In [7]:
num_test_sets = len([col for col in df_day.columns if col.startswith('test_')])
num_test_sets

3

In [8]:
def get_train_i(i):
  return df_day[df_day['train_'+str(i)]]

def get_test_i(i):
  df_test = df_day[df_day['test_'+str(i)]].copy()
  # Replace the temperature colmn by the temperature_forecast_i column
  df_test['temperature'] = df_test['temperature_forecast_'+str(i)]
  return df_test


In [9]:
def get_location_name(df):

  df2 = df.copy()

  df2['location'] = ''
  df2.loc[df2['location_Kaiserstraße'] == 1, 'location'] = 'Kaiserstraße'
  df2.loc[df2['location_Schönbornstraße'] == 1, 'location'] = 'Schönbornstraße'
  df2.loc[df2['location_Spiegelstraße'] == 1, 'location'] = 'Spiegelstraße'

  return df2['location']

In [10]:
def format_dataset_for_nixtla(df):
    df = df.copy()
    # Add column unique_id that is the location name
    df['unique_id'] = get_location_name(df)
    # Rename date to ds and pedestrians_count to y
    df.rename(columns = {'timestamp':'ds', 'pedestrians_count':'y'}, inplace=True)
    df['ds'] = pd.to_datetime(df['ds'])
    # Drop the one hot encoded locations

    # Replace True with 1 and False with 0
    df.replace({True: 1, False: 0}, inplace=True)

    return df[futr_exog_list + ['ds', 'y', 'unique_id']].reset_index(drop=True)

In [11]:
futr_exog_list = ['special_event', 'holidays', 'weekday_Friday', 'weekday_Monday',
       'weekday_Saturday', 'weekday_Sunday', 'weekday_Thursday',
       'weekday_Tuesday', 'weekday_Wednesday', 'month_1', 'month_2',
       'month_3', 'month_4', 'month_5', 'month_6', 'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12',
       'pedestrians_count_7d_lag']


static_feature_list = ['location_Kaiserstraße','location_Schönbornstraße','location_Spiegelstraße']

ml_features_list = futr_exog_list + static_feature_list



In [12]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error


def mae_loss(y_true, qdist):
    """
    Takes all quantile estimations as input and then calculates the MAE loss for the 50% quantile (median)
    The MAE loss is the mean absolute error between the true values and the predicted values.
    ´´´
    """
    return mean_absolute_error(y_true, qdist)




- Der **Pinball-Loss** berechnet den Fehler zwischen den vorhergesagten und den tatsächlichen Werten für jedes Quantil und gewichtet diese Fehler je nach dem zugehörigen Quantil.

- In diesem Fall wird der Pinball-Loss verwendet, weil das Modell nicht nur einen Punktwert (wie bei der Vorhersage des Mittelwerts), sondern verschiedene Quantile der Verteilung der Fußgängerzahlen vorhersagen soll. Das heißt, es geht nicht darum, eine exakte Vorhersage eines bestimmten Werts zu machen, sondern verschiedene Szenarien vorherzusagen (wie hohe oder niedrige Fußgängerzahlen an bestimmten Tagen), die durch die Quantile repräsentiert werden

- Einfache Verlustfunktionen wie der Mean Squared Error (MSE) sind auf Punktvorhersagen (Mittelwerte) beschränkt und nicht geeignet, um Verteilungen oder Unsicherheiten zu modellieren. Der Pinball-Loss hingegen ermöglicht es, die Vorhersagequalität auf verschiedenen Quantilen zu bewerten.


**Ziel:** Diese Funktion vergleicht die Quantilverluste (mit der Pinball-Loss-Funktion) zwischen dem empirischen Modell und dem vorhergesagten Modell. Der durchschnittliche prozentuale Verbesserungswert wird berechnet und gibt an, wie viel besser das Modell im Vergleich zur empirischen Verteilung ist.

- Für die Fußgängerzählergebnisse: wie verhalten sich die Zahlen bei extremen Werten (wie dem 95. Quantil für hohe Fußgängerzahlen) oder bei sehr niedrigen Werten (5. Quantil)? Der Pinball-Loss ist ideal, um solche extremen Szenarien zu bewerten.

- Die Pinball-Loss-Funktion berechnet die Differenz zwischen dem tatsächlichen Wert und dem vorhergesagten Wert für jedes Quantil. Der Verlust wird unterschiedlich gewichtet, je nachdem, ob die tatsächlichen Werte über oder unter dem Quantil liegen.

## LightGBM

In [13]:
import lightgbm as lgb

lgb_mae = []


for i in range(1, num_test_sets + 1):

    df_train = get_train_i(i)
    df_test = get_test_i(i)

    df_test_schönborn = df_test[df_test['location_Schönbornstraße'] == 1]
    df_test_kaiser = df_test[df_test['location_Kaiserstraße'] == 1]
    df_test_spiegel = df_test[df_test['location_Spiegelstraße'] == 1]

    # Define the model with quantiles
    model = lgb.LGBMRegressor(
        objective='quantile',
        alpha=0.5, # alpha Parameter gibt Quantil an
        random_state=42,
        verbose=-1
    )

    # Fit the model
    model.fit(df_train[ml_features_list], df_train['pedestrians_count'])

    # Predict the quantiles

    preds_schönborn = model.predict(df_test_schönborn[ml_features_list])
    preds_kaiser = model.predict(df_test_kaiser[ml_features_list])
    preds_spiegel = model.predict(df_test_spiegel[ml_features_list])

    mae_schönborn = mae_loss(df_test_schönborn['pedestrians_count'], preds_schönborn)
    mae_kaiser = mae_loss(df_test_kaiser['pedestrians_count'], preds_kaiser)
    mae_spiegel = mae_loss(df_test_spiegel['pedestrians_count'], preds_spiegel)

    lgb_mae.append(
      {'mae_schönborn': mae_schönborn, 'mae_kaiser': mae_kaiser, 'mae_spiegel': mae_spiegel}
      )







/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [14]:
pd.DataFrame(lgb_mae)


,mae_schönborn,mae_kaiser,mae_spiegel
0,3885.521684,1521.218024,2838.955020
1,2356.628325,1519.476485,1389.907055
2,5250.755468,2222.054448,2383.950596


In [15]:
pd.DataFrame(lgb_mae).mean()


,0
mae_schönborn,3830.968492
mae_kaiser,1754.249652
mae_spiegel,2204.270891


In [16]:
pd.DataFrame(lgb_mae).to_csv(f'/content/drive/My Drive/BA/lgb_mae_{SHORT_TIME_OR_LONG_TIME}.csv', index=False)
pd.DataFrame(lgb_mae).mean().to_csv(f'/content/drive/My Drive/BA/lgb_mae_mean_{SHORT_TIME_OR_LONG_TIME}.csv', index=True)
#